In [4]:
import os
import time
import pytz
import pandas as pd
import numpy as np
from datetime import datetime


DATA_DIR = '/data/cmpe257-02-fa2019/team-1-meerkats/rsna-intracranial-hemorrhage-detection/'
TIMESTAMP_BEGIN = str(datetime.now(pytz.timezone('America/Los_Angeles'))).replace(" ", "-")
MODEL_NAME = TIMESTAMP_BEGIN + '-ResNet50-conv+head-10k'
WEIGHTS_DIR = '/data/cmpe257-02-fa2019/team-1-meerkats/weights/' + MODEL_NAME + '/'
os.mkdir(WEIGHTS_DIR)

LEARNING_RATES = np.linspace(1e-6,1e-5,10)

!ls $DATA_DIR
INPUT_SHAPE = (224, 224, 3)

stage_2_sample_submission.csv  stage_2_test  stage_2_train  stage_2_train.csv


In [5]:
# dcm processing

def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    #handle the 12 bit values
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def window_and_scale_brain_subdural_soft(dcm):
    
    #window images
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    #soft_img = window_image(dcm, 40, 380)
    bone_img = window_image(dcm, 600, 2800)
    
    #scale images (0-1)
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img + 20) / 200
    bone_img = (bone_img + 800) / 2800
    
    # combine channels
    return np.array([brain_img, subdural_img, bone_img]).transpose(1,2,0)

def old_window_and_scale(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img


def read_trainset(filename=DATA_DIR+"stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        56346,56347,56348,56349,
        56350,56351,1171830,1171831,
        1171832,1171833,1171834,1171835,
        3705312,3705313,3705314,3705315,
        3705316,3705317,3842478,3842479,
        3842480,3842481,3842482,3842483
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

In [6]:
import keras
import cv2
from math import ceil
import numpy as np
import pydicom

np.random.seed(2557)

def _read(path, desired_size):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = window_and_scale_brain_subdural_soft(dcm)
        img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
        
    # Some dcms seem to be corrupted
    except ValueError:
        print('Error while parsing {}'.format(path))
        img = np.ones(desired_size)
    
    return img

class DataGenerator(keras.utils.Sequence):

    def __init__(self, img_dir, image_IDs, labels_df, batch_size, img_size):

        self.image_IDs = image_IDs
        self.labels_df = labels_df
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir

    def __len__(self):
        return int(ceil(len(self.image_IDs) / self.batch_size))

    def __getitem__(self, index):
        
        batch_ids = self.image_IDs[index*self.batch_size:(index+1)*self.batch_size]
        
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, 6))
        
        for i, ID in enumerate(batch_ids):
            X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            Y[i,] = self.labels_df.loc[ID].values
        
        return X, Y

Using TensorFlow backend.


In [7]:
from sklearn.model_selection import ShuffleSplit 

train_df = read_trainset()

# k-fold splitting
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=257).split(train_df.index)
# get indeces for one split
train_idx, valid_idx = next(ss)
train_df_kfold = train_df.iloc[train_idx]
valid_df_kfold = train_df.iloc[valid_idx]


traingen = DataGenerator(img_dir=DATA_DIR+'stage_2_train/',
                         image_IDs=train_df_kfold.index[:10000], #MAGIC
                         labels_df=train_df_kfold[:10000], #MAGIC
                         batch_size=16,
                         img_size=INPUT_SHAPE)

validgen = DataGenerator(img_dir=DATA_DIR+'stage_2_train/',
                         image_IDs=valid_df_kfold.index[:1000], #MAGIC
                         labels_df=valid_df_kfold[:1000], #MAGIC
                         batch_size=16,
                         img_size=INPUT_SHAPE)

In [8]:
# custom loss function
from keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights = np.array([1., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)

# custom performance metric
def correct_diagnoses(y_true, y_pred):
    THRESHOLD = 0.5
    p_thr = K.greater(y_pred, THRESHOLD)
    y_true = K.cast(y_true, dtype='bool')
    
    equals_t = K.equal(p_thr, y_true)
    correct_rows = K.all(equals_t, axis=1)
    correct_rows_int = K.cast(correct_rows, dtype='int32')
    
    return K.sum(correct_rows_int)/K.shape(correct_rows_int)[0]

def correct_positive_diagnoses(y_true, y_pred):
    THRESHOLD = 0.5
    p_thr = K.greater(y_pred, THRESHOLD)
    y_true = K.cast(y_true, dtype='bool')
    
    pos_mask = K.any(y_true, axis=1) #patients with positive diagnoses
    p_thr = p_thr[pos_mask]
    y_true = y_true[pos_mask]
    
    equals_t = K.equal(p_thr, y_true)
    correct_rows = K.all(equals_t, axis=1)
    correct_rows_float = K.cast(correct_rows, dtype='float32')
    
    return K.sum(correct_rows_float)/(K.cast(K.shape(correct_rows_float)[0], dtype='float32')+K.epsilon())

def np_cpd(y_true, pred, thr=0.5): #numpy implementation of correct positive diagnoses
    p_thr = pred > thr

    pos_mask = np.any(y_true, axis=1)

    p_thr = p_thr[pos_mask]
    y_true = y_true[pos_mask]

    p_correct = np.all(p_thr[:len(y_true)] == y_true[:len(p_thr)], axis=1)

    return np.sum(p_correct)/(len(p_thr)+1e-15)

In [9]:
mc = keras.callbacks.ModelCheckpoint(filepath=WEIGHTS_DIR+'epoch={epoch:02d}-valid-loss={val_loss:.2f}.hdf5', monitor='loss', verbose=True, save_best_only=False, save_weights_only=False)
tb = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

In [ ]:
import keras
#from keras.applications.vgg16 import VGG16
from keras.applications import ResNet50
from keras import layers
import numpy as np

#conv_base = VGG16(weights=None, input_shape=INPUT_SHAPE ,include_top=False)
#conv_base.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5') #doesn't work otherwise without internet access

models = []
histories = []

for lr in LEARNING_RATES:
    print(f'trying lr={lr}')
    conv_base = ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    conv_base.trainable = True
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(6, activation='sigmoid'))
    model.name = f'ResNet50-10k-lr={lr}'
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(lr=lr),
        metrics=[correct_positive_diagnoses])
    
    hist = model.fit_generator(traingen,
                    validation_data = validgen,
                    epochs=10,
                    verbose=True,
                    use_multiprocessing=True,
                    workers=4,
                    callbacks=[mc])
    
    histories.append(hist)
    models.append(model)

trying lr=1e-06














/home/013855803/anaconda3/envs/brainenv/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
625/625 [==============================] - 237s 379ms/step - loss: 0.2482 - correct_positive_diagnoses: 0.0049 - val_loss: 0.1726 - val_correct_positive_diagnoses: 0.0000e+00

Epoch 00001: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=01-valid-loss=0.17.hdf5
Epoch 2/10
625/625 [==============================] - 216s 345ms/step - loss: 0.1506 - correct_positive_diagnoses: 0.0204 - val_loss: 0.1594 - val_correct_positive_diagnoses: 0.0000e+00

Epoch 00002: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=02-valid-loss=0.16.hdf5
Epoch 3/10
625/625 [==============================] - 212s 339ms/step - loss: 0.1115 - correct_positive_diagnoses: 0.0936 - val_loss: 0.1538 - val_correct_positive_diagnoses: 0.0000e+00

Epoch 00003: s

/home/013855803/anaconda3/envs/brainenv/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/10
625/625 [==============================] - 245s 392ms/step - loss: 0.2136 - correct_positive_diagnoses: 0.0053 - val_loss: 0.1621 - val_correct_positive_diagnoses: 0.0032

Epoch 00001: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=01-valid-loss=0.16.hdf5

Epoch 2/10
625/625 [==============================] - 211s 337ms/step - loss: 0.1186 - correct_positive_diagnoses: 0.0806 - val_loss: 0.1525 - val_correct_positive_diagnoses: 0.0177

Epoch 00002: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=02-valid-loss=0.15.hdf5
Epoch 3/10
625/625 [==============================] - 225s 360ms/step - loss: 0.0697 - correct_positive_diagnoses: 0.3406 - val_loss: 0.1513 - val_correct_positive_diagnoses: 0.0495

Epoch 00003: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+

...

Epoch 1/10
625/625 [==============================] - 259s 414ms/step - loss: 0.1737 - correct_positive_diagnoses: 0.0232 - val_loss: 0.1320 - val_correct_positive_diagnoses: 0.0304

Epoch 00001: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=01-valid-loss=0.13.hdf5

Epoch 2/10
625/625 [==============================] - 215s 345ms/step - loss: 0.0625 - correct_positive_diagnoses: 0.3930 - val_loss: 0.1527 - val_correct_positive_diagnoses: 0.0688

Epoch 00002: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=02-valid-loss=0.15.hdf5
Epoch 3/10
625/625 [==============================] - 212s 339ms/step - loss: 0.0158 - correct_positive_diagnoses: 0.8029 - val_loss: 0.1631 - val_correct_positive_diagnoses: 0.0463

Epoch 00003: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+

/home/013855803/anaconda3/envs/brainenv/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/10
625/625 [==============================] - 272s 435ms/step - loss: 0.1708 - correct_positive_diagnoses: 0.0292 - val_loss: 0.1351 - val_correct_positive_diagnoses: 0.0331

Epoch 00001: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=01-valid-loss=0.14.hdf5

Epoch 2/10
625/625 [==============================] - 222s 354ms/step - loss: 0.0613 - correct_positive_diagnoses: 0.3939 - val_loss: 0.1412 - val_correct_positive_diagnoses: 0.0780

Epoch 00002: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=02-valid-loss=0.14.hdf5
Epoch 3/10
625/625 [==============================] - 218s 349ms/step - loss: 0.0146 - correct_positive_diagnoses: 0.8162 - val_loss: 0.1668 - val_correct_positive_diagnoses: 0.0648

Epoch 00003: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+

/home/013855803/anaconda3/envs/brainenv/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/10
625/625 [==============================] - 279s 447ms/step - loss: 0.1726 - correct_positive_diagnoses: 0.0270 - val_loss: 0.1310 - val_correct_positive_diagnoses: 0.0204

Epoch 00001: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=01-valid-loss=0.13.hdf5
Epoch 2/10
625/625 [==============================] - 232s 371ms/step - loss: 0.0604 - correct_positive_diagnoses: 0.3896 - val_loss: 0.1488 - val_correct_positive_diagnoses: 0.1008

Epoch 00002: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+head-10k/epoch=02-valid-loss=0.15.hdf5
Epoch 3/10
625/625 [==============================] - 226s 362ms/step - loss: 0.0136 - correct_positive_diagnoses: 0.8095 - val_loss: 0.1715 - val_correct_positive_diagnoses: 0.0738

Epoch 00003: saving model to /data/cmpe257-02-fa2019/team-1-meerkats/weights/2019-12-03-03:44:06.044677-08:00-ResNet50-conv+h

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.ylabel('loss')
plt.xlabel('epoch')
handles = []
for i in range(len(histories)):
    ax, = plt.plot(histories[i].history['loss'], label=f'lr={round(LEARNING_RATES[i],7)}')
    handles.append(ax)
    
plt.legend(handles=handles,loc='best')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.ylabel('validation loss')
plt.xlabel('epoch')
handles = []
for i in range(len(histories)):
    ax, = plt.plot(histories[i].history['val_loss'], label=f'lr={round(LEARNING_RATES[i],7)}')
    handles.append(ax)
    
plt.legend(handles=handles)
plt.show()

In [ ]:
histories[i].history.keys()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.ylabel('CPD')
plt.xlabel('epoch')
handles = []
for i in range(len(histories)):
    ax, = plt.plot(histories[i].history['correct_positive_diagnoses'], label=f'lr={round(LEARNING_RATES[i],7)}')
    handles.append(ax)
    
plt.legend(handles=handles)
plt.show()

# Conclusion
Looks like 1e-5 is th